# **Построение модели**

## **Подключение библиотек**

In [9]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit.Chem import rdFingerprintGenerator
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
import joblib

## **Загрузка данных о ингибиторах**

In [10]:
inhibitors_filepath = "../data/SARS-COV-2/high_inhibitory_compounds/"
inhibitors_nature = pd.read_csv(filepath_or_buffer=inhibitors_filepath + 'nature/processed_data/sars-cov-2_inhibitors_nature.csv')
inhibitors_pubs = pd.read_csv(filepath_or_buffer=inhibitors_filepath + 'pubs/processed_data/sars-cov-2_inhibitors_pubs.csv')
inhibitors_sciencedirect = pd.read_csv(filepath_or_buffer=inhibitors_filepath + 'sciencedirect/processed_data/sars-cov-2_inhibitors_sciencedirect.csv')



inhibitors_nature.drop(columns=['inhibitor_name', 'CC50_µM', 'EC50_µM'], inplace=True, axis=1)
inhibitors_pubs.drop(columns=['compound_name', 'IC50_μM', 'CC50_μM'], inplace=True, axis=1)
inhibitors_sciencedirect.drop(columns=['inhibitor_name', 'IC50_μmol/L', 'CC50_μmol/L'], inplace=True, axis=1)

inhibitors_df = pd.concat([inhibitors_nature, inhibitors_pubs, inhibitors_sciencedirect], axis=0)

inhibitors_df.drop_duplicates(subset='smiles', inplace=True)

print(inhibitors_df.shape)

inhibitors_df.head()

(133, 2)


,smiles,SI
0,C[C@@](C1=CC=CC=C1)(C2=CC=C(C=C2)Cl)OCC[C@H]3C...,7.96
1,CCCCC1=C(C2=CC=CC=C2O1)C(=O)C3=CC(=C(C(=C3)I)O...,10.63
2,CC(CN1C2=CC=CC=C2SC3=CC=CC=C31)CN(C)C,14.21
3,CN1CCN(CC1)CCCOC2=C(C=C3C(=C2)N=CC(=C3NC4=CC(=...,13.58
4,CN(C)CCOC1=CC=C(C=C1)/C(=C(/CCCl)\C2=CC=CC=C2)...,8.92


## **Загрузка данных о соединениях с низкой ингибиторностью**

In [11]:
non_inhibitors_df = pd.read_csv(filepath_or_buffer='../data/SARS-COV-2/low_inhibitory_compounds/processed_data/non_inhibitors_db.csv')
non_inhibitors_df.drop(columns=['ligand_name', 'IC50_nM_parsed'], inplace=True)

complete_data = pd.concat([inhibitors_df, non_inhibitors_df], axis=0)

complete_data.shape

(643, 2)

## **Удаление выбросов**

In [12]:
def remove_outliers(df, columns):
    Q1 = df[columns].quantile(0.25)
    Q3 = df[columns].quantile(0.75)
    IQR = Q3 - Q1
    
    # Определение маски для выбросов
    mask = ~((df[columns] < (Q1 - 1.5 * IQR)) | (df[columns] > (Q3 + 1.5 * IQR))).any(axis=1)
    
    # Возвращаем DataFrame с удалёнными выбросами для указанных столбцов
    return df[mask]



complete_data = pd.concat([remove_outliers(inhibitors_df, ['SI']), non_inhibitors_df], axis=0)

print(inhibitors_df.shape[0] + non_inhibitors_df.shape[0])
print(complete_data.shape[0])


complete_data

643
617


,smiles,SI
0,C[C@@](C1=CC=CC=C1)(C2=CC=C(C=C2)Cl)OCC[C@H]3C...,7.96
1,CCCCC1=C(C2=CC=CC=C2O1)C(=O)C3=CC(=C(C(=C3)I)O...,10.63
2,CC(CN1C2=CC=CC=C2SC3=CC=CC=C31)CN(C)C,14.21
3,CN1CCN(CC1)CCCOC2=C(C=C3C(=C2)N=CC(=C3NC4=CC(=...,13.58
4,CN(C)CCOC1=CC=C(C=C1)/C(=C(/CCCl)\C2=CC=CC=C2)...,8.92
...,...,...
505,Cc1cccc(CN2CCN(CC2)c2nc3ccccc3nc2C)c1,0.00
506,CC(NC(C)c1ccccc1C)C(=O)Nc1ccc2[nH]c(=O)[nH]c2c1,0.00
507,C\C=C\C=C\C(=O)N1CCC(=CC1)c1c[nH]c2ncccc12,0.00
508,CN1CCCC(C1)N(C(C(=O)NCCc1cccc(F)c1)c1cccnc1)C(...,0.00


## **Извлечение следующих признаков из `SMILES`**:
- MolecularWeight
- LogP
- HBD
- HBA
- TPSA
- fingerprint

In [13]:
def extract_morgan_fingerprint(smiles, radius=2, nBits=2048):
    # Преобразуем SMILES в молекулу
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return None
    
    # Создаем генератор Morgan fingerprints
    fpgen = AllChem.GetMorganGenerator(radius=radius, fpSize=nBits)
    
    # Генерируем fingerprint в виде битовой векторной формы
    morgan_fp = fpgen.GetFingerprint(mol)
    
    # Преобразуем результат в массив numpy
    morgan_fp_arr = np.array(morgan_fp)
    
    return morgan_fp_arr

def extract_features(smiles):
    # Преобразование SMILES в молекулу RDKit
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return None
    
    # Вычисление молекулярной массы
    molecular_weight = Descriptors.MolWt(mol)
    
    # Вычисление LogP (коэффициента распределения)
    logP = Descriptors.MolLogP(mol)
    
    # Число водородных доноров (HBD)
    hbd = Descriptors.NumHDonors(mol)
    
    # Число водородных акцепторов (HBA)
    hba = Descriptors.NumHAcceptors(mol)
    
    # Топологическая полярная поверхность (TPSA)
    tpsa = Descriptors.TPSA(mol)
    
    rdkit_gen = rdFingerprintGenerator.GetRDKitFPGenerator(maxPath=5)
    fingerprint = rdkit_gen.GetFingerprint(mol)
    
    

    return {
        'MolecularWeight': molecular_weight,
        'LogP': logP,
        'HBD': hbd,
        'HBA': hba,
        'TPSA': tpsa,
        'fingerprint' : fingerprint
    }

In [14]:
features_df = complete_data['smiles'].apply(extract_features)

features_expanded_df = pd.DataFrame(features_df.tolist())


complete_data.reset_index(drop=True, inplace=True)
features_expanded_df.reset_index(drop=True, inplace=True)

complete_data_with_features = pd.concat([complete_data, features_expanded_df], axis=1, ignore_index=False)

print(complete_data_with_features.dtypes)

complete_data_with_features.head()

smiles              object
SI                 float64
MolecularWeight    float64
LogP               float64
HBD                  int64
HBA                  int64
TPSA               float64
fingerprint         object
dtype: object


,smiles,SI,MolecularWeight,LogP,HBD,HBA,TPSA,fingerprint
0,C[C@@](C1=CC=CC=C1)(C2=CC=C(C=C2)Cl)OCC[C@H]3C...,7.96,343.898,5.10440,0,2,12.47,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,CCCCC1=C(C2=CC=CC=C2O1)C(=O)C3=CC(=C(C(=C3)I)O...,10.63,645.319,6.93620,0,4,42.68,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
2,CC(CN1C2=CC=CC=C2SC3=CC=CC=C31)CN(C)C,14.21,298.455,4.48700,0,3,6.48,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,CN1CCN(CC1)CCCOC2=C(C=C3C(=C2)N=CC(=C3NC4=CC(=...,13.58,530.456,5.19038,1,8,82.88,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
4,CN(C)CCOC1=CC=C(C=C1)/C(=C(/CCCl)\C2=CC=CC=C2)...,8.92,405.969,6.21500,0,2,12.47,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
selective_index = complete_data_with_features['SI']
selective_index = np.array(selective_index)
selective_index = selective_index.flatten()

In [16]:
compounds = []
for smile, mw, logp, hbd, hba, tpsa in complete_data_with_features[['smiles', 'MolecularWeight', 'LogP', 'HBD', 'HBA', 'TPSA']].itertuples(index=False):
    compounds.append((Chem.MolFromSmiles(smile), mw, logp, hbd, hba, tpsa))
rdkit_gen = rdFingerprintGenerator.GetRDKitFPGenerator(maxPath=5)
fingerprints = np.array([rdkit_gen.GetFingerprint(mol) for mol, _, _, _, _, _ in compounds])
properties = np.array([[mw, logp, hbd, hba, tpsa] for _, mw, logp, hbd, hba, tpsa in compounds])


combined_data = np.hstack((fingerprints, properties))

combined_data

array([[ 0.  ,  0.  ,  0.  , ...,  0.  ,  2.  , 12.47],
       [ 0.  ,  1.  ,  0.  , ...,  0.  ,  4.  , 42.68],
       [ 0.  ,  0.  ,  1.  , ...,  0.  ,  3.  ,  6.48],
       ...,
       [ 1.  ,  0.  ,  1.  , ...,  1.  ,  2.  , 48.99],
       [ 1.  ,  0.  ,  0.  , ...,  1.  ,  6.  , 91.57],
       [ 1.  ,  1.  ,  0.  , ...,  2.  ,  5.  , 87.32]])

## **Разделение обработанных данных на тренировочную и тестовую часть**

In [17]:
X_train, X_test, y_train, y_test = train_test_split(combined_data, selective_index, test_size = 0.2)

## **Выбор XGBoost**

В твоей задаче регрессии для предсказания селективного индекса (SI) на основе молекулярных признаков и отпечатков, XGBoost оказался наилучшим выбором по нескольким причинам:

1. **Обработка сложных данных**: Данные включают в себя молекулярные отпечатки (вектор признаков размером 2048) и дополнительные физико-химические свойства (например, молекулярная масса, LogP, HBD и TPSA). Такие данные могут быть высокоразмерными и разреженными, и XGBoost хорошо справляется с подобными задачами за счет своей способности обрабатывать большие массивы признаков и улавливать сложные нелинейные зависимости.

2. **Устойчивость к шуму и выбросам**: Деревья решений, на которых основан XGBoost, хорошо справляются с данными, которые могут содержать шум или выбросы. В данном случае это особенно важно, так как в химических и биологических данных могут присутствовать аномальные значения, и модель должна быть к ним устойчива.

3. **Контроль переобучения**: XGBoost имеет встроенные механизмы для предотвращения переобучения, такие как регуляризация и возможность контролировать сложность модели через параметры, такие как `max_depth`, `gamma` и другие. В данном проекте важна хорошая обобщающая способность модели, а XGBoost, благодаря гибкой настройке гиперпараметров, позволяет контролировать баланс между переобучением и недообучением.

4. **Эффективность и масштабируемость**: XGBoost хорошо оптимизирован для работы с большими наборами данных и может быть использован на более сложных молекулярных данных благодаря высокоэффективной реализации. Это делает его быстрым и производительным даже на больших выборках.

5. **Лучшие результаты по тестам**: После проведения серии экспериментов и тестов, включая настройку гиперпараметров через RandomizedSearchCV, XGBoost показал лучшие результаты по сравнению с другими моделями. В частности, метрика Mean Absolute Error (MAE) для XGBoost оказалась наименьшей, что говорит о том, что модель на основе XGBoost более точно предсказывает селективный индекс (SI) для соединений.

Таким образом, благодаря способности XGBoost эффективно обрабатывать сложные, разреженные и высокоразмерные данные, а также демонстрировать наилучшие результаты по метрикам качества, эта модель стала оптимальным выбором для решения данной задачи регрессии.

In [18]:
xgb_model = XGBRegressor(random_state=1)

# Определение параметров для перебора
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10, 20],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'min_child_weight': [1, 2, 3, 4],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3]
}

# Настройка RandomizedSearchCV
randomized_search_xgb = RandomizedSearchCV(estimator=xgb_model, 
                                            param_distributions=param_dist, 
                                            n_iter=500,  # Количество итераций
                                            cv=5,  # Количество фолдов для кросс-валидации
                                            random_state=42,
                                            error_score='raise')

## **Настройка гиперпараметров**

In [19]:
print(X_train.shape)
randomized_search_xgb.fit(X_train, y_train)

(493, 2053)


RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints...
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=1, ...),
                   n_iter=500,
                   param_distributions={'colsample_bytree': [0.5, 0.7, 1.0],
                                        'gamma': [0, 0.1, 0.2, 0.3],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 5, 7, 10, 20],
                                        'min_child_weight': [1, 2, 3, 4],
                                        'n_estimators': [100, 200, 300, 500],
                                        'subsample': [0.5, 0.7, 1.0]},
                   random_state=42)

In [20]:
tuned_xgb_model = randomized_search_xgb.best_estimator_
mae_xgb = mean_absolute_error(y_test, tuned_xgb_model.predict(X_test))
print(f"MAE of tuned XGB model: {mae_xgb}")

MAE of tuned XGB model: 2.23073477202005


## **Сохранение модели**

In [21]:
joblib.dump(tuned_xgb_model, 'sars-cov-2_SI_predictive_model.joblib')

['sars-cov-2_SI_predictive_model.joblib']